<a href="https://colab.research.google.com/github/TongHere/Tweet-Sentiment-Analyse-/blob/main/TopicModeling_NegativeTweet_ipyn_LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import Libraries

import os
from itertools import chain
import json
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from pandas.io.json import json_normalize
import seaborn as sns
import nltk

In [ ]:
path = r"/content/tweets_ws22_v1.json"

with open(path) as f:
   
      dump =json.loads(f.read())
      org_df = pd.json_normalize(dump)
      org_df
    
final_df = pd.DataFrame() 

for i in range(0,len(dump)):
    temp_topic_df = pd.json_normalize(dump[i]['labels']['topic'])
    temp_sentiment_df = pd.json_normalize(dump[i]['labels']['sentiment'])
    temp_topic_df.rename(columns={'id':'topic_id','tweet_id':'topic_tweet_id','user_id':'topic_user_id','ground_truth':'topic_ground_truth'}, inplace = True)
    temp_sentiment_df.rename(columns={'id':'sentiment_id','tweet_id':'sentiment_tweet_id','user_id':'sentiment_user_id','ground_truth':'sentiment_ground_truth'}, inplace = True)
    df_join_topic_sent = temp_topic_df.join(temp_sentiment_df)
    final_df = final_df.append(df_join_topic_sent)
    #final_df = pd.concat([org_df,Joined_df],axis = 1)
    
    
cols = final_df.columns
org_df[cols] = final_df[cols].values


data =  org_df.drop(columns=['source_created_at','longitude','latitude','labels.topic','labels.sentiment','topic_tweet_id','sentiment_tweet_id','author_id','source','topic_user_id','topic_ground_truth','language','id','source_id','relevant','sentiment_ground_truth','sentiment_id','topic_id','sentiment_user_id'])
data

,text,topic,sentiment
0,@DSisourath The Thameslink core between London...,service,negative
1,@DulwichHistory Loving the complaint about peo...,delays,negative
2,@SW_Help .And yet you have no toilets on some ...,toilets,negative
3,@SW_Help you have no toilets on some of your t...,toilets,negative
4,@SpeedySticks007 @MrNeilJH @TLRailUK @christia...,seats,neutral
...,...,...,...
16944,Haha oh man the audio corruption on @TLRailUK ...,announcements,neutral
16945,@TLRailUK SweetIs there a plug to charge my ph...,plugs,neutral
16946,@TLRailUK now there are far fewer commuters ha...,tables,neutral
16947,@geofftech I am voting for Thameslink. 1. in a...,toilets,neutral


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gensim
from gensim import corpora, models
import pandas as pd
import nltk
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords 
import string
from gensim.models import CoherenceModel
import math
from nltk.corpus import wordnet
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import nltk

nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
df = data[(data.sentiment == "negative") & (data.topic == "delays")]
df

,text,topic,sentiment
1,@DulwichHistory Loving the complaint about peo...,delays,negative
5,@TLRailUK @SW_Help @SouthernRailUK @TfL Yyyiou...,delays,negative
9,@TLRailUK announced a cancelled train > 2 mins...,delays,negative
25,"""Congestion"" is NOT a reason for delay, it's a...",delays,negative
26,"""Govia Thameslink Railway (GTR) is facing a £5...",delays,negative
...,...,...,...
16725,Thameslink at Bedford; en route to Elstree for...,delays,negative
16750,The 7.15 wrh to lbg train is ALWAYS delayed by...,delays,negative
16801,Well done @TLRailUK again 😠 8:16 SAC to Bright...,delays,negative
16807,What a total joke on starting my hen do @LNER ...,delays,negative


In [ ]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [ ]:
df = data[(data.sentiment == "negative")]
df

,text,topic,sentiment
0,@DSisourath The Thameslink core between London...,service,negative
1,@DulwichHistory Loving the complaint about peo...,delays,negative
2,@SW_Help .And yet you have no toilets on some ...,toilets,negative
3,@SW_Help you have no toilets on some of your t...,toilets,negative
5,@TLRailUK @SW_Help @SouthernRailUK @TfL Yyyiou...,delays,negative
...,...,...,...
16928,@geofftech I’ve done Thameslink even if he Ser...,seats,negative
16934,"Come on Thameslink, despite the fact I don’t f...",seats,negative
16935,@TLRailUK You’ve never done it. I’m here now a...,service,negative
16941,"@GWRHelp @jamiebishop123 Hi @gwrhelp, I'm sick...",seats,negative


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
descriptions = df['text']

In [ ]:
documents = []
texts = []
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english')).union({'data', 'dataset', 'model'})
digits = set(string.digits)
for description in descriptions.iteritems():
    document_str = str(description[1]).lower()
    document_str = document_str.translate(str.maketrans('', '', string.punctuation))
    token_words = word_tokenize(document_str)
    pos_tagged = pos_tag(token_words)
    tokens = [(token, get_wordnet_pos(tag)) for token, tag in pos_tagged]
    lemma_tokens = [lemmatizer.lemmatize(token, tag) for token, tag in tokens]
    document = [w for w in lemma_tokens if (not w in stop_words) and (not w.isdigit()) ]
    documents.append(document)
    texts.append(' '.join(document))
print(documents[0])



['dsisourath', 'thameslink', 'core', 'london', 'st', 'pancras', 'london', 'blackfriar', 'rush', 'hour', 'onlybut', 'cab', 'rubbish', 'rarely', 'work']


In [ ]:
dictionary = corpora.Dictionary(documents)

document_term = [dictionary.doc2bow(document) for document in documents]

In [ ]:
ldamodel = models.ldamodel.LdaModel(document_term, num_topics=20, id2word = dictionary)

In [ ]:
ldamodel.print_topics(num_words=5)

[(0,
  '0.049*"tlrailuk" + 0.033*"delay" + 0.031*"train" + 0.014*"service" + 0.010*"one"'),
 (1,
  '0.044*"tlrailuk" + 0.039*"get" + 0.032*"delay" + 0.028*"train" + 0.014*"stop"'),
 (2,
  '0.063*"due" + 0.054*"delay" + 0.034*"thameslink" + 0.033*"train" + 0.031*"minute"'),
 (3,
  '0.053*"train" + 0.052*"delay" + 0.041*"thameslink" + 0.031*"due" + 0.022*"london"'),
 (4,
  '0.055*"tlrailuk" + 0.055*"’" + 0.043*"delay" + 0.026*"day" + 0.022*"every"'),
 (5,
  '0.044*"delay" + 0.030*"tlrailuk" + 0.028*"redhill" + 0.027*"service" + 0.024*"’"'),
 (6,
  '0.054*"tlrailuk" + 0.052*"delay" + 0.022*"train" + 0.021*"min" + 0.018*"repay"'),
 (7,
  '0.045*"delay" + 0.044*"service" + 0.025*"thameslink" + 0.023*"tlrailuk" + 0.016*"cancel"'),
 (8,
  '0.054*"train" + 0.036*"delay" + 0.031*"time" + 0.029*"’" + 0.027*"tlrailuk"'),
 (9,
  '0.049*"delay" + 0.048*"due" + 0.046*"thameslink" + 0.041*"train" + 0.040*"app"'),
 (10,
  '0.045*"delay" + 0.024*"tlrailuk" + 0.021*"departure" + 0.015*"train" + 0.014*"t

In [ ]:
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 5.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=6dd26d210e2c4a7eb718ea179481b0481d55f0077beda8104a5b32ff7361bd94
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=e533665fbfc583b2dd32266ddc61e4cdee9c505fb008d729e30c2a576cb9c624
  Stored in directory: /root/.cache/pip/wheels/42/56/cc/4a8bf86613aafd5b7f1b310477667c1fca5c51c3ae4124a003
Successfully built pyLDAvis sklearn


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [ ]:
!pip install pyLDAvis

import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, document_term, dictionary)
vis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
19    -0.174485 -0.050358       1        1  16.545617
18     0.112483 -0.192786       2        1   8.012580
17    -0.128647 -0.027625       3        1   7.802284
1     -0.152519 -0.049461       4        1   7.458510
4     -0.107365 -0.082015       5        1   6.538852
13    -0.128215 -0.032992       6        1   6.511422
9     -0.027024 -0.043315       7        1   4.896196
11     0.126773 -0.139947       8        1   4.824074
14    -0.121501 -0.039885       9        1   4.768572
5     -0.157064  0.096526      10        1   4.197942
0      0.118465 -0.073008      11        1   4.121145
16     0.086265 -0.006142      12        1   3.924809
12     0.173247  0.064983      13        1   3.364120
8     -0.136256  0.149834      14        1   3.338141
3      0.178634  0.088930      15        1   3.194913
6      0.048363 -0.008815      16        1   2.413103
2     -0.008137  0.032483      17        1   2.337253
10     0.017899  0.187231      18        1   2.090851
7      0.084028  0.082928      19        1   1.995673
15     0.195057  0.043435      20        1   1.663944, topic_info=           Term         Freq        Total Category  logprob  loglift
34        train  6912.000000  6912.000000  Default  30.0000  30.0000
12   thameslink  2895.000000  2895.000000  Default  29.0000  29.0000
377           ’  6009.000000  6009.000000  Default  28.0000  28.0000
11           st   928.000000   928.000000  Default  27.0000  27.0000
75        delay  3469.000000  3469.000000  Default  26.0000  26.0000
..          ...          ...          ...      ...      ...      ...
75        delay    40.541466  3469.126087  Topic20  -4.3755  -0.3534
57      service    30.495585  2200.025884  Topic20  -4.6603  -0.1827
34        train    32.413653  6912.095307  Topic20  -4.5993  -1.2665
12   thameslink    28.904381  2895.338848  Topic20  -4.7139  -0.5109
61     tlrailuk    27.888732  7060.129950  Topic20  -4.7496  -1.4380

[1352 rows x 6 columns], token_table=       Topic      Freq    Term
term                          
3915       1  0.950728  10mins
1451       1  0.340101     1st
1451       3  0.637689     1st
1451       8  0.014171     1st
1746       1  0.288405     2nd
...      ...       ...     ...
10520     14  0.903236   🤷🏻‍♂️
4037      13  0.950557       🥰
6042      14  0.918141       🥵
6860       8  0.213007       🥶
6860      18  0.724225       🥶

[4257 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[20, 19, 18, 2, 5, 14, 10, 12, 15, 6, 1, 17, 13, 9, 4, 7, 3, 11, 8, 16])

In [ ]:
log_perplexity = ldamodel.log_perplexity(document_term)

In [ ]:
print('Perplexity:', math.exp(log_perplexity))

Perplexity: 0.0010305886840580075


In [ ]:
coherence_model_lda = CoherenceModel(model=ldamodel, texts=documents, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence score:', coherence_lda)

Coherence score: 0.34529060158447644
